In [2]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB



################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from pandas import read_csv
from searchUtils import findPatternExt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



# Parse Spotify

In [3]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
from spotifyCharts import spotifyCharts

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
sd = spotifyData(minYear=1, maxYear=2021)
sd.parse()
#sd.saveChartData()

In [ ]:
sc = spotifyCharts()
sc.getChartsByRank(2)

In [ ]:
# Full 

In [ ]:
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)

In [ ]:
sd = spotifyData(minYear=1, maxYear=2021)
sd.setDBRenames(manDB)
sd.setMultiDBRenames(multimanDB)
sd.setChartUsage(rank=[0,1])
sd.setFullChartData()
sd.setArtistAlbumData()
sd.saveArtistAlbumData()
sd.saveFullChartData()

# Download New Spotify Data

In [4]:
def downloadURL(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    print("Now Downloading {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
        
    return data, response.getcode()

# Global Parameters

In [ ]:
from datetime import datetime

In [8]:
countries = ['sa', 'ie', 'ar', 'no', 'ma', 'in', 'cl', 'hu', 'ch', 'ru', 'lu', 'pt', 'bg', 'jp', 'hn', 'se', 'vn', 'ro', 'hk', 'ec', 'cr', 'nz', 
             'id', 'gb', 'fr', 'sk', 'pe', 'pa', 'lt', 'br', 'ua', 'gr', 'it', 'es', 'fi', 'eg', 'cz', 'uy', 'il', 'py', 'do', 'is', 'at', 'dk',
             'sg', 'ni', 'ca', 'be', 'mx', 'tr', 'de', 'co', 'tw', 'us', 'ae', 'gt', 'pl', 'sv', 'th', 'nl', 'ee', 'ph', 'bo', 'au', 'my', 'za', 'lv']

from pandas import date_range
from datetime import datetime
start = '12/23/2016'
start = '3/1/2021'

end   = datetime.now().strftime("%Y-%m-%d")
end   = "2021-03-20"
datevals = [x.strftime('%Y-%m-%d') for x in list(date_range(start=start, end=end, freq='W-THU'))]
datevals.reverse()
print(datevals)

['2021-03-18', '2021-03-11', '2021-03-04']


# Regional Data (Top 2000)

In [9]:
from time import sleep
import urllib
from fsUtils import isFile
from ioUtils import saveFile

for country in countries:    
    print("\n\n")
    print("==========>",country,"<=========")
    nDownload = 0
    errs = 0
    for i,dateval in enumerate(datevals):
        if errs > 10:
            break
        if nDownload > 3000:
            break
        if i == len(datevals)-1:
            break
        start = datevals[i+1]
        end   = datevals[i]
        url      = "https://spotifycharts.com/regional/{0}/weekly/{1}--{2}/download".format(country, start, end)
        #savename = "/Users/tgadfort/Documents/code/charts/spotify/
        savename = "/Volumes/Piggy/Charts/data/spotify/categories/regional-{0}-weekly-{1}--{2}.p".format(country, start, end)
        if isFile(savename):
            continue

        data, code = downloadURL(url)
        if False:
            try:
                data, code = downloadURL(url)
            except:
                errs += 1
                continue
            
        saveFile(idata=data, ifile=savename)
        sleep(3.5)
        nDownload += 1




==========> sa <=========
Now Downloading https://spotifycharts.com/regional/sa/weekly/2021-03-11--2021-03-18/download


HTTPError: HTTP Error 404: Not Found

# Viral Data (Top 50)

In [ ]:
from time import sleep
import urllib
from fsUtils import isFile
from ioUtils import saveFile

for country in countries:
    print("\n\n")
    print("==========>",country,"<=========")
    nDownload = 0
    errs = 0
    for i,dateval in enumerate(datevals):
        if errs > 10:
            break
        if nDownload > 3000:
            break
        if i == len(datevals)-1:
            break
        start = datevals[i+1]
        end   = datevals[i]
        url      = "https://spotifycharts.com/viral/{0}/weekly/{1}--{2}/download".format(country, start, start)
        #2021-01-14--2021-01-14
        #url      = "https://spotifycharts.com/regional/{0}/weekly/{1}--{2}/download".format(country, start, end)
        #savename = "/Users/tgadfort/Documents/code/charts/spotify/
        savename = "/Volumes/Piggy/Charts/data/spotifyviral/categories/viral-{0}-weekly-{1}--{2}.p".format(country, start, start)
        if isFile(savename):
            continue

        try:
            data, code = downloadURL(url)
        except:
            errs += 1
            sleep(2)
            continue
            
        saveFile(idata=data, ifile=savename)
        sleep(3.5)
        nDownload += 1

In [ ]:
from spotifyViral import spotifyViralData

In [ ]:
sp = spotifyViralData()
sp.parse()

In [ ]:
countries = ['sa', 'ie', 'ar', 'no', 'ma', 'in', 'cl', 'hu', 'ch', 'ru', 'lu', 'pt', 'bg', 'jp', 'hn', 'se', 'vn',
             'ro','hk', 'ec', 'cr', 'nz', 'id', 'gb', 'fr', 'sk', 'pe', 'pa', 'lt', 'br', 'ua', 'gr', 'it', 'es',
             'fi', 'eg', 'cz', 'uy', 'il', 'py', 'do', 'is', 'at', 'dk', 'sg', 'ni', 'ca', 'be', 'mx', 'tr', 'de',
             'co', 'tw', 'us', 'ae', 'gt', 'pl', 'sv', 'th', 'nl', 'ee', 'ph', 'bo', 'au', 'my', 'za', 'lv']
names = ["Saudi Arabia", "Ireland", "Argentina", "Norway", "India", "Morocco", "Chile", "Hungary", "Switzerland", "Russia", "Luxembourg", "Portugal", "Bulgaria", "Japan", "Honduras", "Sweden", "Vietnam",
        "Romania", "Hong Kong", "Ecuador", "Costa Rica", "New Zealand", "Indonesia", "United Kingdom", "France", "Slovakia", "Peru", "Panama", "Lithuania", "Brazil", "Ukraine", "Greece", "Italy", "Spain",
        "Finland", "Egypt", "Czech Republic", "Uraguay", "Isreal", "Paraguay", "Dominican Republic", "Iceland", "Austria", "Denmark", "Singapore", "Nicaragua", "Canada", "Belgium", "Mexico", "Turkey", "Germany",
        "Colombia", "Taiwan", "United States", "United Arab Emirates", "Guatemala", "Poland", "El Salvador", "Thailand", "Netherlands", "Estonia", "Philippines", "Bolivia", "Australia", "Malaysia", "South Africa", "Latvia"]
comb = dict(zip(countries, names))

In [ ]:
print(comb)

In [ ]:
for abbr,name in comb.items():
    print("        self.{0: <19} = ['regional-{1}-weekly']".format(name.replace(" ", "").lower(), abbr))

In [ ]:
chartRanks = {}
chartRanks[0] = ['unitedstates', 'canada']
chartRanks[1] = ['unitedkingdom', 'australia', 'ireland', 'newzealand']
chartRanks[2] = ['germany', 'austria', 'switzerland']
chartRanks[3] = ['italy', 'france', 'spain', 'netherlands', 'luxembourg']
chartRanks[4] = ['finland', 'iceland', 'belgium', 'czechrepublic', 'hungary']
chartRanks[5] = ['denmark', 'sweden', 'norway']
chartRanks[6] = ['isreal', 'turkey', 'saudiarabia', 'unitedarabemirates', 'egypt', 'morocco']
chartRanks[7] = ['russia', 'ukraine', 'bulgaria', 'latvia', 'estonia', 'lithuania', 'slovakia', 'romania', 'poland']
chartRanks[8] = ['mexico', 'costarica', 'panama', 'dominicanrepublic', 'guatemala', 'nicaragua', 'honduras', 'elsalvador']        
chartRanks[9] = ['colombia', 'argentina', 'peru', 'paraguay', 'uraguay', 'chile', 'bolivia', 'ecuador']
chartRanks[10] = ['japan', 'hongkong', 'vietnam', 'taiwan', 'thailand', 'malaysia', 'singapore']
chartRanks[11] = ['philippines', 'greece', 'indonesia', 'india', 'southafrica']
chartRanks[12] = ['brazil', 'portugal']
chartRanks[13] = ['global']

In [ ]:
from listUtils import getFlatList
ranks = getFlatList(chartRanks.values())

In [ ]:
countriesList = set([name.replace(" ", "").lower() for name in comb.values()])
ranksList = set(ranks)
countriesList.symmetric_difference(ranksList)

In [ ]:
len(countriesList)

In [ ]:
len(ranksList)

In [ ]:
ranksList.difference(countriesList)